In [ ]:
import pandas as pd
import numpy as np
import re
import string
from data_helper import * 

In [ ]:
x_train,y_train,x_test,y_test=load_data_and_labels("./data/train.csv")

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [3]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helper
from text_cnn import TextCNN
# from tensorflow.python.compiler import tensorrt as trt
import csv
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.utils import to_categorical

ModuleNotFoundError: No module named 'text_cnn'

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from distutils.version import LooseVersion, StrictVersion
import gensim

model =  './data/baomoi.window2.vn.model.bin'

if os.path.isfile(model):
    print("Loading word2vec model ...")
if LooseVersion(gensim.__version__) >= LooseVersion("1.0.1"):
    word2vec_model = KeyedVectors.load_word2vec_format(model, binary=True)
else:
    print("wrong")
    word2vec_model = Word2Vec.load_word2vec_format(model, binary=True)

In [ ]:
class Relevance:
    def __init__(self, word2vec):
        self.w2v_model = word2vec

    def vectorize(self, doc: str) -> np.ndarray:
        doc = doc.lower()
        words = [w for w in doc.split(" ")]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                pass
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        relevance = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(relevance)):
            return 0
        return relevance

    def calculate_similarity(self, source_doc, target_docs=None):
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        print(source_vec)
        results = []
        for doc in target_docs:
            print(doc)
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            results.append({"score": sim_score, "doc": doc})
        return results

In [ ]:
def clean_str(target):
    """
    Tokenization/string cleaning for all datasets.
    """
    target = re.sub(r"\'s", " \'s", target.lower())
    target = re.sub(r"\'ve", " \'ve", target)
    target = re.sub(r"n\'t", " n\'t", target)
    target = re.sub(r"\'re", " \'re", target)
    target = re.sub(r"\'d", " \'d", target)
    target = re.sub(r"\'ll", " \'ll", target)
    target = re.sub(r",", " , ", target)
    target = re.sub(r"!", " ! ", target)
    target = re.sub(r"\(", " \( ", target)
    target = re.sub(r"\)", " \) ", target)
    target = re.sub(r"\?", " \? ", target)
    target = re.sub(r"—", " ", target)
    target = re.sub(r"\s{2,}", " ", target)
    target = re.sub(r"\s{2,}", " ", target)
    target = re.sub(r"xem", " ", target)
    target = re.sub(r"thêm", " ", target)
    target = re.sub(r"watch", " ", target)
    target = re.sub(r"more", " ", target)
    target= re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", target)
    target = re.sub(r'[^\w]', ' ', target)
    return target.translate(str.maketrans('', '', string.punctuation)).strip()

In [ ]:
ds = Relevance(word2vec_model)

In [ ]:
data=pd.read_csv("d_comment.csv")["content (S)"].values
post=pd.read_csv("d_post.csv")["content (S)"].values[0]

In [ ]:
for i in data:
    print(ds.calculate_similarity(post,i))

In [59]:
from pyvi import ViTokenizer


In [61]:
ViTokenizer.tokenize(clean_str(post)).split(' ')

['thấy',
 'nhiều',
 'anh_em',
 'quan_tâm',
 'việc',
 'thông_thạo',
 'nâng_cấp',
 'vạch',
 'vạch',
 'với',
 'thần',
 'hoả',
 'nên',
 'admin',
 'giải_đáp_đây',
 'ctrl',
 'có',
 'thông_thạo',
 'là',
 'nâng_cấp',
 'được',
 'mở',
 'khoá',
 'bộ',
 'thần',
 'hoả',
 'thì',
 'được',
 'vạch',
 'hiện',
 'mới',
 'có',
 'bộ',
 'nên',
 'được',
 'tối_đa',
 'vạch',
 'xem']

In [5]:
from CocCocTokenizer import PyTokenizer
T = PyTokenizer(load_nontone_data=True)

In [6]:
matrix_vec_train=[]
for i in x_train:
    matrix_vec_train.append(makeFeatureVec(T.word_tokenize(i, tokenize_option=0),word2vec_model,100).reshape(100*300))
matrix_vec_train=np.array(matrix_vec_train)

In [7]:
matrix_vec_test=[]
for i in x_test:
    matrix_vec_test.append(makeFeatureVec(T.word_tokenize(i, tokenize_option=0),word2vec_model,300).reshape(100*300))
matrix_vec_test=np.array(matrix_vec_train)

In [8]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
maxlen = max(len(x) for x in matrix_vec_train) # longest text in train set

In [10]:
sequence_length = matrix_vec_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5
EMBEDDING_DIM=300
inputs = Input(shape=(sequence_length,))
embedding_layer = Embedding(len(word2vec_model.vocab),
                           EMBEDDING_DIM,
                           weights=[word2vec_model.vectors])
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
model = Model(inputs, output)


In [11]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30000)        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30000, 300)   131716800   input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 30000, 300, 1 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 29998, 1, 100 90100       reshape_1[0][0]                  
____________________________________________________________________________________________

In [12]:
print('Shape of X train and X validation tensor:', matrix_vec_train.shape,matrix_vec_test.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_test.shape)

Shape of X train and X validation tensor: (1000, 30000) (1000, 30000)
Shape of label train and validation tensor: (1000, 2) (500, 2)


In [17]:
np.expand_dims(matrix_vec_train, 1)

array([[[ 0.24589975,  0.77774334, -1.2496276 , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.7005746 ,  1.3664861 , -1.6598337 , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.01931669,  1.5146191 , -0.95038223, ...,  0.        ,
          0.        ,  0.        ]],

       ...,

       [[ 1.5678059 ,  0.9919252 , -1.4614412 , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.01931669,  1.5146191 , -0.95038223, ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.79908675,  1.0495341 , -0.29474068, ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32)

In [28]:
len(matrix_vec_train)

1000

In [30]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(matrix_vec_train, y_train, batch_size=10, epochs=10, verbose=1, validation_data=(matrix_vec_test, y_test),
         callbacks=callbacks) 

/opt/anaconda3/envs/coccoc_word_tokenize/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


InvalidArgumentError:  indices[1,3] = -1 is not in [0, 439056)
	 [[node embedding_1/embedding_lookup (defined at /opt/anaconda3/envs/coccoc_word_tokenize/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_2385]

Function call stack:
keras_scratch_graph


In [23]:
y_train.shape

(1000, 2)

In [ ]:
# Define CNN architecture
# del model
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(len(word2vec_model.vocab),
                           word2vec_model.vectors.shape[1],
                           weights=[word2vec_model.vectors],
                           input_length=maxlen))
model.add(layers.Conv2D(100, kernel_size = (3, 300), padding='valid', data_format="channels_last", input_shape = (1000, 30000), activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())



In [ ]:
# Fit model
history = model.fit(matrix_vec_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(matrix_vec_test, y_test),
                    batch_size=64)
loss, accuracy = model.evaluate(matrix_vec_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(matrix_vec_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))